# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/facebook_friends.gml")

In [3]:
print(nx.info(g))

Graph with 329 nodes and 1954 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [6]:
from myTruss import mappingAndRelabeling

In [7]:
g_relabled = mappingAndRelabeling(g)

In [8]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 329 nodes and 1954 edges
Graph with 329 nodes and 1954 edges


<ipython-input-8-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-8-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [9]:
g = g_relabled

# Infer community structure with Louvain

In [10]:
nx.__version__

'2.8.4'

In [11]:
import pandas as pd
from os.path  import exists
import infomap as im

In [12]:
im.__version__

'2.3.0'

In [13]:
G = g

In [14]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [15]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [16]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [17]:
results_file = "./results/facebookfriends-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/facebookfriends-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/facebookfriends-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/facebookfriends-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [18]:
len(mec_ranks)

329

In [19]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[74,
 39,
 95,
 80,
 90,
 83,
 68,
 87,
 29,
 130,
 127,
 31,
 66,
 138,
 69,
 139,
 86,
 143,
 15,
 94,
 99,
 22,
 23,
 154,
 26,
 30,
 81,
 135,
 141,
 4,
 18,
 105,
 93,
 157,
 34,
 19,
 10,
 11,
 96,
 164,
 244,
 1,
 20,
 64,
 61,
 40,
 79,
 140,
 243,
 27,
 2,
 12,
 72,
 82,
 17,
 126,
 5,
 6,
 13,
 98,
 92,
 129,
 155,
 128,
 0,
 3,
 7,
 16,
 147,
 146,
 45,
 250,
 175,
 57,
 75,
 89,
 76,
 131,
 156,
 28,
 266,
 265,
 8,
 233,
 165,
 67,
 133,
 32,
 245,
 136,
 14,
 163,
 242,
 235,
 179,
 251,
 85,
 241,
 177,
 169,
 78,
 172,
 134,
 272,
 70,
 204,
 170,
 171,
 124,
 44,
 174,
 9,
 168,
 84,
 21,
 137,
 248,
 247,
 106,
 249,
 220,
 49,
 88,
 97,
 73,
 111,
 194,
 205,
 206,
 132,
 191,
 159,
 142,
 46,
 187,
 189,
 218,
 55,
 167,
 280,
 181,
 33,
 253,
 178,
 113,
 217,
 214,
 285,
 254,
 232,
 252,
 119,
 216,
 257,
 50,
 303,
 215,
 24,
 65,
 276,
 103,
 35,
 121,
 211,
 212,
 219,
 47,
 209,
 213,
 271,
 274,
 279,
 234,
 100,
 238,
 41,
 43,
 240,
 112,
 306,
 166,
 180,

In [20]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [21]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [22]:
dict_map_eq_centrality

{74: 329,
 39: 328,
 95: 327,
 80: 326,
 90: 325,
 83: 324,
 68: 323,
 87: 322,
 29: 321,
 130: 320,
 127: 319,
 31: 318,
 66: 317,
 138: 316,
 69: 315,
 139: 314,
 86: 313,
 143: 312,
 15: 311,
 94: 310,
 99: 309,
 22: 308,
 23: 307,
 154: 306,
 26: 305,
 30: 304,
 81: 303,
 135: 302,
 141: 301,
 4: 300,
 18: 299,
 105: 298,
 93: 297,
 157: 296,
 34: 295,
 19: 294,
 10: 293,
 11: 292,
 96: 291,
 164: 290,
 244: 289,
 1: 288,
 20: 287,
 64: 286,
 61: 285,
 40: 284,
 79: 283,
 140: 282,
 243: 281,
 27: 280,
 2: 279,
 12: 278,
 72: 277,
 82: 276,
 17: 275,
 126: 274,
 5: 273,
 6: 272,
 13: 271,
 98: 270,
 92: 269,
 129: 268,
 155: 267,
 128: 266,
 0: 265,
 3: 264,
 7: 263,
 16: 262,
 147: 261,
 146: 260,
 45: 259,
 250: 258,
 175: 257,
 57: 256,
 75: 255,
 89: 254,
 76: 253,
 131: 252,
 156: 251,
 28: 250,
 266: 249,
 265: 248,
 8: 247,
 233: 246,
 165: 245,
 67: 244,
 133: 243,
 32: 242,
 245: 241,
 136: 240,
 14: 239,
 163: 238,
 242: 237,
 235: 236,
 179: 235,
 251: 234,
 85: 233,
 24

In [23]:
# Write Centrality
with open('./MapEquationCentralityResults/OSN-FacebookFriends/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [24]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/OSN-FacebookFriends/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [25]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True